In [ ]:
'''
Author: Patrick Rudolph
Date: 12/16/19
Description: assess data for outliers and potential errors
'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

path = __import__('00_setup')

In [ ]:
#import tread measurements
filepath = path.input + 'dataset.xlsx'
tread = pd.read_excel(filepath)

In [ ]:
tread.shape

In [ ]:
tread.columns

In [ ]:
# duplicates
tread.drop_duplicates(subset=['TestId','VehicleNumber','Branding','Insp']).shape[0] - tread.shape[0]

In [ ]:
# missing values
cols =['TestId','Branding','VehicleNumber','Axle','TirePosition','Month, Day, Year of Insp Date','RTD','Avg. Odometer','IP']
tread[cols].isnull().sum()

Missing values are all from 'lost or missing' records

In [ ]:
# check removals
tread.groupby('RemovalReason')['RemovalReason'].size()

AD: mileage is taken at time of removal, even if tire is replaced before field measurement

In [ ]:
# vehilces per test
tread.groupby(['TestId','Pattern'])['VehicleNumber'].nunique()

3370 and 3380 are Sprinters<br/>
3381 and 3382 are Promasters<br/>
10 vehicles per tire-model combo

In [ ]:
# tires per vehicle
tread.drop_duplicates(subset=['TestId','VehicleNumber','Branding']).groupby('VehicleNumber')['VehicleNumber'].count().sort_values(ascending = False)

AD: only should be 4 tires per vehicle. >4 is result of mounted tire on different vehicle than specified. Can assume first measurement is where the tire should be<br/>
Fix: overwrite mount vehicle with value from first inspection

In [ ]:
# sort by location-vehicle-tire
tread.sort_values(['TestId','VehicleNumber','Branding','Month, Day, Year of Insp Date'], inplace = True)

In [ ]:
# reset index
tread.reset_index(inplace = True, drop = True)

In [ ]:
# create prev record fields
tread['tread_depth_prev'] = tread['RTD'].shift(1)
tread['odometer_prev'] = tread['Avg. Odometer'].shift(1)

# differentials
tread['treadwear'] = tread['tread_depth_prev'] - tread['RTD']
tread['miles'] = tread['Avg. Odometer'] - tread['odometer_prev']

In [ ]:
# check negative tread depths
cond1 = (tread['TestId'] == tread['TestId'].shift(1))
cond2 = (tread['VehicleNumber'] == tread['VehicleNumber'].shift(1))
cond3 = (tread['Branding'] == tread['Branding'].shift(1))
cond4 = (tread['treadwear'] < 0)

tread[cond1 & cond2 & cond3 & cond4]

AD: small negative tread measurements could result from tire variations in wear and placement of measurement<br/>
Fix: set these to 0

In [ ]:
# check odometer is consistent for same vehicle-date
odometer_count = tread.groupby(['VehicleNumber','Month, Day, Year of Insp Date','Avg. Odometer']).size()
odometer_count = odometer_count.reset_index().rename(columns={0:'count'})
odometer_count[odometer_count.duplicated(subset = ['VehicleNumber','Month, Day, Year of Insp Date'], keep = False)]

Inconsistent odometer readings for same vehicle-date<br/>
Lower readings appear to be from removals<br/>
Assuming measurement was taken prior to date indicated, no fix needed

In [ ]:
# first record flag
cond1 = (tread['TestId'] != tread['TestId'].shift(1))
cond2 = (tread['VehicleNumber'] != tread['VehicleNumber'].shift(1))
cond3 = (tread['Branding'] != tread['Branding'].shift(1))

first_record = (cond1 | cond2 | cond3)

In [ ]:
# descreasing odometer
pd.options.display.max_columns = None
cond1 = (tread['Avg. Odometer'] < tread['Avg. Odometer'].shift(1))
tread.loc[(~first_record & cond1)]

Odometer went down<br/>
Later date and lower tread depths, so assuming error<br/>
Fix: set to missing

In [ ]:
# check odometer = 0
tread[tread['Avg. Odometer'] == 0]

Record from misassigned vehcile at mount<br/>
Inspection 1 appears to have correct mount odometer<br/>
Fix: overwrite odometer at inspection 0 with mount odometer value of inspection 1

In [ ]:
# set first values to 0
tread.loc[first_record, 'miles'] = 0
tread.loc[first_record, 'treadwear'] = 0

In [ ]:
# cumulative miles and wear
tread['cuml_miles'] = tread.groupby(['TestId','VehicleNumber','Branding'], as_index = False)['miles'].cumsum()
tread['cuml_treadwear'] = tread.groupby(['TestId','VehicleNumber','Branding'], as_index = False)['treadwear'].cumsum()

In [ ]:
metrics = ['RTD','Avg. Odometer','IP','treadwear','miles','cuml_miles','cuml_treadwear']
for metric in metrics:
    print(metric)
    print(tread[metric].describe())
    tread[metric].hist()
    plt.show()

Outlier: pressure = 728<br/>
Pressure = 0, values are all for removals<br/>
Fix: set values over 150 or 0 to missing

In [ ]:
tread.plot(figsize = (15,5), x = 'miles', y = 'treadwear', kind = 'scatter');

In [ ]:
tread.plot(figsize = (15,5), x = 'cuml_miles', y = 'RTD', kind = 'scatter');

In [ ]:
tread.plot(figsize = (15,5), x = 'cuml_miles', y = 'cuml_treadwear', kind = 'scatter');